## Finite difference validation of elastic solid derivatives

In [ ]:
import sys; sys.path.append('..')
import energy, tensors, mesh, elastic_solid
import numpy as np
from numpy.linalg import norm
from matplotlib import pyplot as plt
import fd_validation

In [ ]:
def testMesh(dim, deg):
    if (dim == 2): return mesh.Mesh('../../misc/examples/meshes/square_hole.off', deg)
    if (dim == 3): return mesh.Mesh('../../misc/examples/meshes/ball.msh', deg)

In [ ]:
def getOrthoTensor(dim):
    if (dim == 3):
        et = tensors.ElasticityTensor3D()
        et.setOrthotropic(1, 1.5, 2, 0.3, 0.05, -0.03, 0.51, 0.73, 0.82)
    if (dim == 2):
        et = tensors.ElasticityTensor2D()
        et.setOrthotropic(1, 1, 0.3, 0.5)
    return et

In [ ]:
def getNeoHookean(dim): return energy.NeoHookeanYoungPoisson(dim, 200, 0.35)
def getLinearElastic(dim): return energy.IsotropicLinearElastic(dim, 200, 0.35)
def getCRIsoLinearElastic(dim): return energy.CorotatedIsotropicLinearElastic(dim, 200, 0.35)
def getStVK(dim):
    e = energy.StVenantKirchhoffAutoProjected(getOrthoTensor(dim))
    e.projectionEnabled = False # Of course the projected Hessian does not pass FD tests.
    return e
def getCROrthoLinearElastic(dim): return energy.CorotatedLinearElastic(getOrthoTensor(dim))
def getCRIsoHessProj(dim):
    psi = energy.IsoCRLEWithHessianProjection(dim, 200, 0.35)
    psi.projectionEnabled = False # Of course the projected Hessian does not pass FD tests.
    return psi

In [ ]:
def getPerturb(F):
    return np.random.uniform(-1, 1, F.shape)

def genPlots(getEnergy):
    def test(dim, deg):
        m = testMesh(dim, deg)
        obj = elastic_solid.ElasticSolid(m, getEnergy(dim))
        curr = obj.getVars()
        delta_x = getPerturb(curr)
        obj.setVars(curr + 1e-2 * getPerturb(curr)) # Evaluate away from the zero-gradient undeformed configuration 
        fd_validation.gradConvergencePlotRaw(obj, perturb=delta_x)
        fd_validation.hessConvergencePlotRaw(obj, perturb=delta_x)
        plt.grid()
        plt.legend()
        name = obj.__class__.__name__
        try: name += "\n(Isotropic)" if obj.getEnergyDensity(0).isIsotropic() else "\n(Orthotropic)"
        except: pass
        plt.title(name)
    fig = plt.figure(figsize=(20, 4))
    for dim in [2, 3]:
        for deg in [1, 2]:
            plt.subplot(1, 4, 2 * (dim - 2) + deg)
            test(dim, deg)
    plt.tight_layout()

In [ ]:
for ge in [getNeoHookean, getLinearElastic, getCRIsoLinearElastic, getCROrthoLinearElastic,
           getStVK, getCRIsoHessProj]:
    genPlots(ge)